In [ ]:
computer = 'win'

if computer == 'mac':
    str_chromedriver = '../../../chromedriver'
elif computer == 'win':
    str_chromedriver = '../../../chromedriver.exe'
else:
    print('？？？')
    exit(0)

# = = = = = = = = = = = = = = =

from gevent import monkey
monkey.patch_all(thread=False)
from gevent.queue import Queue
import gevent

import sys
sys.path.append('../../00.module')
import Proxy
import UA

# = = = = = = = = = = = = = = =

import pandas as pd

df_menu = pd.read_excel('./part.xlsx',
                        header=0,
                        dtype=str).fillna('')

list_url = df_menu['Vehicle_Url'].tolist()

print('总数量：' + str(len(list_url)))
print()

# = = = = = = = = = = = = = = =

df = pd.DataFrame()

# = = = = = = = = = = = = = = =

from selenium.webdriver import ChromeOptions

option = ChromeOptions()
option.add_experimental_option('useAutomationExtension', False)
option.add_argument('blink-settings=imagesEnabled=false')
option.add_argument('--disable-blink-features=AutomationControlled')
option.add_argument('--disable-dev-shm-usage')
option.add_argument('--disable-gpu')
option.add_experimental_option('excludeSwitches', ['enable-automation'])
option.add_argument('--headless')
option.add_argument('--no-sandbox')
option.add_argument('--proxy-server=' + Proxy.get_Proxy_Selenium())
option.add_argument('user-agent=' + UA.get_User_Agent())
option.page_load_strategy = 'eager'

# = = = = = = = = = = = = = = =

from selenium.webdriver import Chrome
from selenium.webdriver.chrome.service import Service

# = = = = = = = = = = = = = = =

import time

# = = = = = = = = = = = = = = =

from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.common.by import By

# = = = = = = = = = = = = = = =

from datetime import datetime

# = = = = = = = = = = = = = = =

work = Queue()
for url in list_url:
    work.put_nowait(url)

def crawler():
    global df
    
    while not work.empty():
        url = work.get_nowait()
        
        # = = = = = = = = = = = = = = =
        
        status = 'error'
        for _ in range(31):
            try:
                browser = Chrome(service=Service(str_chromedriver), options=option)

                browser.maximize_window()
                
                # = = = = = = = = = = = = = = =
                
                browser.get(url)

                time.sleep(3)
                
                # = = = = = = = = = = = = = = =
                
                dict_vehicle = {}
                list_position = []
                list_application = []
                while True:
                    wait = WebDriverWait(browser, 60)
                    wait.until(EC.presence_of_element_located((By.XPATH, '//tr[@class="detail-app-row"]')))
                    list_vehicle = browser.find_elements(by=By.XPATH, value='//tr[@class="detail-app-row"]')

                    for vehicle in list_vehicle:
                        year = int(vehicle.find_elements(by=By.XPATH, value='./td')[0].text.strip())
                        make = vehicle.find_elements(by=By.XPATH, value='./td')[1].text.strip()
                        model = vehicle.find_elements(by=By.XPATH, value='./td')[2].text.strip()

                        if make not in dict_vehicle:
                            dict_vehicle[make] = {model: [year]}
                        else:
                            if model not in dict_vehicle[make]:
                                dict_vehicle[make][model] = [year]
                            else:
                                dict_vehicle[make][model].append(year)

                        list_position.append(vehicle.find_elements(by=By.XPATH, value='./td')[4].text.strip())
                        list_application.append(vehicle.find_elements(by=By.XPATH, value='./td')[5].text.strip())

                    if len(browser.find_elements(by=By.XPATH, value='//a[@aria-label="Next"]')) == 0:
                        break
                    else:
                        browser.find_element(by=By.XPATH, value='//a[@aria-label="Next"]').click()

                list_vehicle = []
                for make in dict_vehicle:
                    for model in dict_vehicle[make]:
                        list_vehicle.append(make + ' ' + model + ' ' + str(min(dict_vehicle[make][model])) + '-' + str(max(dict_vehicle[make][model])))

                vehicle = '\n'.join(list_vehicle)
                    
                position = '\n'.join(list(set(list_position)))
                application = '\n'.join(list(set(list_application)))

                # = = = = = = = = = = = = = = =
                
                df_temp = pd.DataFrame([{'status': 'ok',
                                         'Vehicle_Url': url,
                                         'Vehicle': vehicle,
                                         'Position': position,
                                         'Application': application}])
                
                browser.quit()
                
                status = 'ok'
                
                break
                
            except:
                browser.quit()
                time.sleep(3)
                continue
                
                
        # = = = = = = = = = = = = = = =

        if status == 'error':
            df_temp = pd.DataFrame([{'status': 'error',
                                     'Vehicle_Url': url}])

        # = = = = = = = = = = = = = = =

        df = pd.concat([df, df_temp], ignore_index=True).fillna('')

        print(url +  '  <->  [' + status + '] - 剩余数量：' + str(work.qsize()))

# = = = = = = = = = = = = = = =

list_task = []
for _ in range(20):
    task = gevent.spawn(crawler)
    list_task.append(task)
gevent.joinall(list_task)

print()
print('爬虫输出中')
df.to_excel('./vehicle_' + datetime.now().strftime('%Y%m%d_%H%M%S') + '.xlsx', index=False)
print()
print('爬虫结束')